Initialiation 

Just importing necessary packages and connecting to the database. 

In [3]:
from telnetlib import STATUS
import psycopg2
import pandas as pd
import sys
from tqdm import tqdm

password = 'PAiac14-'
host = 'greenplum01.corral.tacc.utexas.edu'
port = 5432
database = 'uthealth'

username = 'piacobelli'
connection = psycopg2.connect(
    host = host, 
    database = database, 
    user = username, 
    port = port, 
    password = password, 
    keepalives = 1, keepalives_idle = 100
)
connection.autocommit = True

C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Row Counts

Creating an initial table to compare the row counts, member ID counts, and claim ID counts between the raw data and the data warehouse.

In [2]:
tableCreateQuery = ''' 
DROP TABLE IF EXISTS qa_reporting.dw_iqva_pharmacy_claims_counts;

CREATE TABLE qa_reporting.dw_iqva_pharmacy_claims_counts
(
    data_source BPCHAR(4),
    calendar_year INT,
    table_src TEXT,
    dw_row_count BIGINT,
    src_row_count BIGINT,
    row_count_diff BIGINT,
    row_count_diff_percentage FLOAT,
    dw_uth_clm_id_count BIGINT,
    dw_src_clm_id_count BIGINT,
    src_clm_count BIGINT,
    clm_count_diff BIGINT,
    clm_count_percentage FLOAT,
    dw_uth_mbr_id_count BIGINT,
    dw_src_mbr_id_count BIGINT,
    src_mbr_count BIGINT,
    mbr_count_diff BIGINT,
    mbr_count_percentage FLOAT,
    date_generated DATE
);
'''

with connection.cursor() as cursor:
    cursor.execute(tableCreateQuery)

In [5]:
with connection.cursor() as cursor:
    countInsertQuery = f'''
        INSERT INTO qa_reporting.dw_iqva_pharmacy_claims_counts
        (
            calendar_year, 
            table_src, 
            dw_row_count, 
            dw_uth_clm_id_count, 
            date_generated
        )
        SELECT 
            year, 
            table_id_src, 
            COUNT(*), 
            COUNT(distinct uth_rx_claim_id), 
            current_date
        FROM dw_staging.iqva_pharmacy_claims a
        GROUP BY 1,2;
    '''
    
    cursor.execute(countInsertQuery)

In [7]:
with connection.cursor() as cursor:
    clmSrcUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_pharmacy_claims_counts b
        SET dw_src_clm_id_count = count
        FROM (
            SELECT 
                year, 
                table_id_src, 
                COUNT(DISTINCT rx_claim_id_src) as count 
            FROM dw_staging.iqva_pharmacy_claims
            GROUP BY 1,2
        ) a
        WHERE a.year = b.calendar_year
        AND b.table_src = a.table_id_src;
    '''

    cursor.execute(clmSrcUpdateQuery)

In [9]:
with connection.cursor() as cursor:

    mbrCountUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_pharmacy_claims_counts b
        SET dw_uth_mbr_id_count = count
        FROM (
            SELECT 
                year, 
                table_id_src, 
                COUNT(DISTINCT uth_member_id) AS count 
            FROM dw_staging.iqva_pharmacy_claims
            GROUP BY 1, 2
        ) a
        WHERE a.year = b.calendar_year
        AND b.table_src = a.table_id_src; 
    '''

    cursor.execute(mbrCountUpdateQuery)

In [ ]:
with connection.cursor() as cursor:

    mbrSrcUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_pharmacy_claims_counts b
        SET dw_src_mbr_id_count = count
        FROM (
            SELECT 
                year, 
                table_id_src, 
                COUNT(distinct member_id_src) AS count 
            FROM dw_staging.iqva_pharmacy_claims
            GROUP BY 1, 2
        ) a
        WHERE a.year = b.calendar_year
        AND b.table_src = a.table_id_src;
    '''

    cursor.execute(mbrSrcUpdateQuery)

In [3]:
with connection.cursor() as cursor:
    pharmClmsQuery = '''
        WITH iqva_claims as (
            SELECT 
                year, 
                pat_id, 
                derv_claimno
            FROM dev.pi_iqvia_derv_claimno_new_all_yr
            WHERE new_rectype = 'P'
        ),
        iqva_claims_counts as (
            SELECT 
                year AS calendar_year, 
                COUNT(*) as src_row_count, 
                COUNT(DISTINCT pat_id) src_mbr_count,
                COUNT(DISTINCT derv_claimno) src_clm_count
            FROM iqva_claims
            GROUP BY 1
        )
        UPDATE qa_reporting.dw_iqva_pharmacy_claims_counts a
        SET src_row_count = b.src_row_count,
            row_count_diff = a.dw_row_count - b.src_row_count,
            row_count_diff_percentage = 100. * abs(a.dw_row_count - b.src_row_count) / b.src_row_count,
            src_clm_count = b.src_clm_count,
            clm_count_diff = a.dw_uth_clm_id_count - b.src_clm_count,
            clm_count_percentage = 100. * abs(a.dw_uth_clm_id_count - b.src_clm_count) / b.src_clm_count,
            src_mbr_count = b.src_mbr_count,
            mbr_count_diff = a.dw_uth_mbr_id_count - b.src_mbr_count,   
            mbr_count_percentage = 100. * abs(a.dw_uth_mbr_id_count - b.src_mbr_count) / b.src_mbr_count
        FROM iqva_claims_counts b
        WHERE a.calendar_year = b.calendar_year;
    '''

    cursor.execute(pharmClmsQuery)
    
pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_pharmacy_claims_counts', con=connection).sort_values(['row_count_diff_percentage', 'clm_count_percentage', 'mbr_count_percentage'])

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_54844\1377390166.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_pharmacy_claims_counts', con=connection).sort_values(['row_count_diff_percentage', 'clm_count_percentage', 'mbr_count_percentage'])


,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
0,None,2023,claims,35562781,35562781,0,0.0,35552153,35552153,35552153,0,0.0,4203615,4203615,4203615,0,0.0,2024-04-10
1,None,2020,claims,87808398,87808398,0,0.0,87465548,87465548,87465548,0,0.0,6123609,6123609,6123609,0,0.0,2024-04-10
2,None,2021,claims,92510269,92510269,0,0.0,92497792,92497792,92497792,0,0.0,6334633,6334633,6334633,0,0.0,2024-04-10
3,None,2019,claims,95244910,95244910,0,0.0,93827512,93827512,93827512,0,0.0,6971462,6971462,6971462,0,0.0,2024-04-10
4,None,2018,claims,101778996,101778996,0,0.0,99933383,99933383,99933383,0,0.0,7056513,7056513,7056513,0,0.0,2024-04-10
5,None,2022,claims,96069141,96069141,0,0.0,96059834,96059834,96059834,0,0.0,6387922,6387922,6387922,0,0.0,2024-04-10
6,None,2017,claims,160251083,160251083,0,0.0,156463843,156463843,156463843,0,0.0,12247226,12247226,12247226,0,0.0,2024-04-10
7,None,2015,claims,199237796,199237796,0,0.0,193473413,193473413,193473413,0,0.0,14325896,14325896,14325896,0,0.0,2024-04-10
8,None,2006,claims,219292009,219292009,0,0.0,216053048,216053048,216053048,0,0.0,16288112,16288112,16288112,0,0.0,2024-04-10
9,None,2016,claims,215313142,215313142,0,0.0,209408302,209408302,209408302,0,0.0,14861066,14861066,14861066,0,0.0,2024-04-10


NDC

Creating a table to count NDCs. Refernce tables do not appear to correspond to NDC ciodes for IQVIA, so the percentage difference will be a little bit high.

In [3]:
ndcQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_ndc_count;
    SELECT 
        year, 
        ndc, 
        COUNT(*)
    INTO qa_reporting.dw_iqva_ndc_count
    FROM dw_staging.iqva_pharmacy_claims
    GROUP BY 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(ndcQuery)

In [4]:
ndcDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_ndc_count;', con=connection)
print(ndcDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_53108\529822536.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ndcDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_ndc_count;', con=connection)


        year          ndc  count
0       2019  50428399766      5
1       2021  67544039960     15
2       2009  00074463701     50
3       2022  00597039013     57
4       2023  58487003061      1
...      ...          ...    ...
888216  2011  00149071001   1504
888217  2014  68382011505   6177
888218  2022  79854013082      3
888219  2009  54868399801      1
888220  2007  00440709130      1

[888221 rows x 3 columns]


In [7]:
ndcDf.groupby(['year'])['count'].sum()

year
2006    219292009
2007    271604546
2008    288199059
2009    280869296
2010    251623423
2011    251858364
2012    232882378
2013    216145055
2014    216523903
2015    199237796
2016    215313142
2017    160251083
2018    101778996
2019     95244910
2020     87808398
2021     92510269
2022     96069141
2023     35562781
Name: count, dtype: int64

In [14]:
ndcAntiJoinQuery = '''
    SELECT a.*
    FROM qa_reporting.dw_iqva_ndc_count a
    LEFT JOIN reference_tables.ndc_tier_map b ON a.ndc = b.ndc_code
    WHERE b.ndc_code IS NULL;
'''

missingNdcDf = pd.read_sql(ndcAntiJoinQuery, con=connection)
print(missingNdcDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_53108\229000462.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  missingNdcDf = pd.read_sql(ndcAntiJoinQuery, con=connection)


        year          ndc  count
0       2013  49769026216      1
1       2017  43353021904      1
2       2007  37205038760      1
3       2018  51552107202      3
4       2008  51927382400      5
...      ...          ...    ...
245927  2015  11673044908      1
245928  2013  08080430101      1
245929  2017  33261089301      1
245930  2017  36800049078      5
245931  2021  30768060693     42

[245932 rows x 3 columns]


In [15]:
missingNdcDf.groupby('year')['count'].sum()

year
2006    3822418
2007    4544660
2008    5107742
2009    5096461
2010    4394779
2011    4353625
2012    4505342
2013    4713279
2014    7661008
2015    8129398
2016    9134689
2017    6403502
2018    4243070
2019    3528331
2020    2285713
2021    3363649
2022    2807379
2023     904736
Name: count, dtype: int64

In [16]:
ndcCompDf = pd.DataFrame(
    {
        'overall_count': ndcDf.groupby('year')['count'].sum(),
        'invalid_count': missingNdcDf.groupby('year')['count'].sum(),
        'valid_count': ndcDf.groupby('year')['count'].sum() - missingNdcDf.groupby('year')['count'].sum()
    }
)
ndcCompDf.loc[ndcCompDf['valid_count'].isna(),'valid_count'] = ndcCompDf.loc[ndcCompDf['valid_count'].isna(),'overall_count']
ndcCompDf['valid_count'] =  ndcCompDf['valid_count'].astype(int)
ndcCompDf['invalid_to_valid_percent'] = 100. * ndcCompDf['invalid_count'] / ndcCompDf['valid_count']
ndcCompDf

,overall_count,invalid_count,valid_count,invalid_to_valid_percent
year,,,,
2006,219292009,3822418,215469591,1.773994
2007,271604546,4544660,267059886,1.701738
2008,288199059,5107742,283091317,1.804274
2009,280869296,5096461,275772835,1.848065
2010,251623423,4394779,247228644,1.777617
2011,251858364,4353625,247504739,1.759007
2012,232882378,4505342,228377036,1.972765
2013,216145055,4713279,211431776,2.229220
2014,216523903,7661008,208862895,3.667960


Days Supply

Getting the min, max and average days supply from the raw data and the data warehouse to compare. 

In [11]:
dayssupCreateQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_rx_days_supply;

    CREATE TABLE qa_reporting.dw_iqva_rx_days_supply
    (
        data_source BPCHAR(4),
        year INT,
        dw_min_days_supply NUMERIC,
        dw_median_days_supply NUMERIC,
        dw_max_days_supply NUMERIC,
        dw_avg_days_supply NUMERIC,
        src_min_days_supply NUMERIC,
        src_median_days_supply NUMERIC,
        src_max_days_supply NUMERIC,
        src_avg_days_supply NUMERIC,
        min_days_supply_diff NUMERIC,
        median_days_supply_diff NUMERIC,
        max_days_supply_diff NUMERIC,
        avg_days_supply_diff NUMERIC
    );
'''

with connection.cursor() as cursor:
    cursor.execute(dayssupCreateQuery)


In [ ]:
with connection.cursor() as cursor:
    dayssupInsertQuery = '''
        INSERT INTO qa_reporting.dw_iqva_rx_days_supply
        (
            year, 
            dw_min_days_supply
        )
        SELECT 
            year,
            min(days_supply)
        FROM dw_staging.iqva_pharmacy_claims
        GROUP BY 1;
    '''

    cursor.execute(dayssupInsertQuery)

In [ ]:
with connection.cursor() as cursor: 

    
    medDayssupUpdateQuery = '''
        UPDATE qa_reporting.dw_iqva_rx_days_supply a
        SET dw_median_days_supply = n
        FROM (
            SELECT 
                year, 
                MEDIAN(days_supply) n
            FROM dw_staging.iqva_pharmacy_claims
            GROUP BY 1
        ) b
        WHERE a.year = b.year
    '''

    cursor.execute(medDayssupUpdateQuery)

In [ ]:
with connection.cursor() as cursor: 
    
    maxDayssupUpdateQuery = '''
        UPDATE qa_reporting.dw_iqva_rx_days_supply a
        SET dw_max_days_supply = n
        FROM (
            SELECT 
                year, 
                MAX(days_supply) n
            FROM dw_staging.iqva_pharmacy_claims
            GROUP BY 1
        ) b
        WHERE a.year = b.year
    '''

    cursor.execute(maxDayssupUpdateQuery)


In [17]:
with connection.cursor() as cursor: 
    
    avgDayssupQuery = '''
        UPDATE qa_reporting.dw_iqva_rx_days_supply a
        SET dw_avg_days_supply = n
        FROM (
            SELECT 
                year, 
                AVG(days_supply) n
            FROM dw_staging.iqva_pharmacy_claims
            GROUP BY 1
        ) b
        WHERE a.year = b.year
    '''

    cursor.execute(avgDayssupQuery)

In [18]:
with connection.cursor() as cursor:
    iqviaClaimsQuery ='''
        DROP TABLE IF EXISTS dev.pi_iqva_rx_daysupp;

        CREATE TABLE dev.pi_iqva_rx_daysupp
        WITH (
            appendonly=true, 
            orientation=row, 
            compresstype=zlib, 
            compresslevel=5 
        )
        AS
        SELECT 
            'iqvia' as data_source, 
            year, 
            pat_id, 
            derv_claimno, 
            dayssup
        FROM dev.pi_iqvia_derv_claimno_new_all_yr
        WHERE rectype = 'P'
        DISTRIBUTED BY (derv_claimno);
    '''

    cursor.execute(iqviaClaimsQuery)

In [19]:
with connection.cursor() as cursor:
    minDayssupQuery = '''
        UPDATE qa_reporting.dw_iqva_rx_days_supply a
        SET src_min_days_supply = n
        FROM (
            SELECT 
                year, 
                min(dayssup::NUMERIC) n
            FROM dev.pi_iqva_rx_daysupp
            GROUP BY 1
        ) b
        WHERE a.year = b.year;
    '''

    cursor.execute(minDayssupQuery)

In [20]:
with connection.cursor() as cursor:

    medDayssupQuery = '''
        UPDATE qa_reporting.dw_iqva_rx_days_supply a
        SET src_median_days_supply = n
        FROM (
            SELECT 
                year, 
                MEDIAN(dayssup::NUMERIC) n
            FROM dev.pi_iqva_rx_daysupp
            GROUP BY 1
        ) b
        WHERE a.year = b.year
    '''

    cursor.execute(medDayssupQuery)

In [21]:
with connection.cursor() as cursor:

    maxDayssupQuery = '''
        UPDATE qa_reporting.dw_iqva_rx_days_supply a
        SET src_max_days_supply = n
        FROM (
            SELECT 
                year, 
                MAX(dayssup::NUMERIC) n
            FROM dev.pi_iqva_rx_daysupp
            GROUP BY 1
        ) b
        WHERE a.year = b.year
    '''

    cursor.execute(maxDayssupQuery)

In [22]:
with connection.cursor() as cursor:

    avgDayssupQuery = '''
        UPDATE qa_reporting.dw_iqva_rx_days_supply a
        SET src_avg_days_supply = n
        FROM (
            SELECT 
                year, 
                AVG(dayssup::NUMERIC) n
            FROM dev.pi_iqva_rx_daysupp
            GROUP BY 1
        ) b
        WHERE a.year = b.year
    '''

    cursor.execute(avgDayssupQuery)

In [4]:
with connection.cursor() as cursor:
    dayssupCompQuery = '''
        UPDATE qa_reporting.dw_iqva_rx_days_supply
        SET min_days_supply_diff = dw_min_days_supply - src_min_days_supply,
        median_days_supply_diff = dw_median_days_supply - src_median_days_supply,
        max_days_supply_diff = dw_max_days_supply - src_max_days_supply,
        avg_days_supply_diff = dw_avg_days_supply - src_avg_days_supply;
    '''

    cursor.execute(dayssupCompQuery)

dayssupDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_rx_days_supply WHERE data_source IS NOT NULL;', con=connection)
print(dayssupDf)

   data_source  year  dw_min_days_supply  dw_median_days_supply  \
0         iqva  2008              -365.0                   30.0   
1         iqva  2018                 0.0                   30.0   
2         iqva  2020                 0.0                   30.0   
3         iqva  2021                 0.0                   30.0   
4         iqva  2014              -999.0                   30.0   
5         iqva  2009              -180.0                   30.0   
6         iqva  2019                 0.0                   30.0   
7         iqva  2016              -750.0                   30.0   
8         iqva  2006              -186.0                   30.0   
9         iqva  2013              -196.0                   30.0   
10        iqva  2011              -180.0                   30.0   
11        iqva  2012              -252.0                   30.0   
12        iqva  2017                 0.0                   30.0   
13        iqva  2007              -365.0                   30.

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_49888\3108280713.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dayssupDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_rx_days_supply WHERE data_source IS NOT NULL;', con=connection)


In [5]:
dayssupDf[['year', 'min_days_supply_diff', 'median_days_supply_diff', 'max_days_supply_diff', 'avg_days_supply_diff']]

,year,min_days_supply_diff,median_days_supply_diff,max_days_supply_diff,avg_days_supply_diff
0,2008,0.0,0.0,0.0,0.186070
1,2018,0.0,0.0,0.0,-0.076569
2,2020,0.0,0.0,0.0,-0.038802
3,2021,0.0,0.0,0.0,-0.008682
4,2014,0.0,0.0,0.0,0.156276
5,2009,0.0,0.0,0.0,0.167051
6,2019,0.0,0.0,0.0,-0.075229
7,2016,0.0,0.0,0.0,0.001649
8,2006,0.0,0.0,0.0,0.144372
9,2013,0.0,0.0,0.0,0.226209


In [6]:
connection.cursor().execute('DROP TABLE IF EXISTS dev.pi_medicare_texas_rx_daysupp;')

Dispense As Written

Getting counts for dispense as written from the raw data and the data warehouse to compare them.

In [21]:
dasTableCreateQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_rx_daw_counts;

    CREATE TABLE qa_reporting.dw_iqva_rx_daw_counts
    (
        data_source BPCHAR(4),
        year INT,
        table_src TEXT,
        dispensed_as_written TEXT,
        dw_count INT,
        src_count INT,
        count_diff INT,
        count_diff_percent NUMERIC
    );
'''

with connection.cursor() as cursor:
    cursor.execute(dasTableCreateQuery)

In [22]:
dasInsertQuery = '''
    INSERT INTO qa_reporting.dw_iqva_rx_daw_counts
    (
        data_source, 
        year,
        table_src, 
        dispensed_as_written, 
        dw_count
    )
    SELECT 
        data_source, 
        year, 
        table_id_src, 
        dispensed_as_written, 
        COUNT(*)
    FROM dw_staging.iqva_pharmacy_claims
    GROUP BY 1,2,3,4;
'''

with connection.cursor() as cursor:
    cursor.execute(dasInsertQuery)

In [24]:
dasClaimsQuery = '''
    WITH rx_daw_count AS (
        SELECT 
            year, 
            'claims' as table_src, 
            CASE WHEN derv_claimno IS NULL THEN '00' ELSE daw END AS dawind, 
            COUNT(*)
        FROM dev.pi_iqvia_derv_claimno_new_all_yr
        WHERE rectype = 'P'
        GROUP BY 1,2,3
    )
    UPDATE qa_reporting.dw_iqva_rx_daw_counts a
    SET src_count = b.count,
    count_diff = dw_count - b.count,
    count_diff_percent = 100. * abs(dw_count - b.count) / b.count
    FROM rx_daw_count b
    WHERE a.year = b.year AND a.dispensed_as_written = b.dawind AND a.table_src = b.table_src;
'''

with connection.cursor() as cursor:
    cursor.execute(dasClaimsQuery)

dasDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_rx_daw_counts', con=connection).sort_values('count_diff_percent')
print(dasDf)

    data_source  year table_src dispensed_as_written   dw_count  src_count  \
18         iqva  2011    claims                    6      11785    11785.0   
83         iqva  2023    claims                    3         67       67.0   
145        iqva  2006    claims                    4      31649    31649.0   
149        iqva  2008    claims                    1    6752068  6752068.0   
152        iqva  2006    claims                    1     356050   356050.0   
..          ...   ...       ...                  ...        ...        ...   
13         iqva  2016    claims                 None  194027988        NaN   
14         iqva  2006    claims                 None  210493590        NaN   
15         iqva  2019    claims                 None   83383701        NaN   
16         iqva  2007    claims                 None  257431941        NaN   
17         iqva  2020    claims                 None   79159025        NaN   

     count_diff  count_diff_percent  
18          0.0          

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_54844\2093048940.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dasDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_rx_daw_counts', con=connection).sort_values('count_diff_percent')
